## Major League Baseball: Data base Design 

This project will be based on Major League Baseball games statistics. We will work with three datasets and build a database with them.


In [1]:
!ls

appearance_type.csv    game_log.csv	    park_codes.csv
Basics.ipynb	       game_log_fields.txt  person_codes.csv
database_design.ipynb  mlb.db		    team_codes.csv


In [2]:
import pandas as pd
import numpy as np
import sqlite3

In [3]:
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [4]:
game_log = pd.read_csv("game_log.csv", dtype=str)

In [5]:
game_log[:5]

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,h_score,length_outs,day_night,completion,forefeit,protest,park_id,attendance,length_minutes,v_line_score,h_line_score,v_at_bats,v_hits,v_doubles,v_triples,v_homeruns,v_rbi,v_sacrifice_hits,v_sacrifice_flies,v_hit_by_pitch,v_walks,v_intentional_walks,v_strikeouts,v_stolen_bases,v_caught_stealing,v_grounded_into_double,v_first_catcher_interference,v_left_on_base,v_pitchers_used,v_individual_earned_runs,v_team_earned_runs,v_wild_pitches,v_balks,v_putouts,v_assists,v_errors,v_passed_balls,v_double_plays,v_triple_plays,h_at_bats,h_hits,h_doubles,h_triples,h_homeruns,h_rbi,h_sacrifice_hits,h_sacrifice_flies,h_hit_by_pitch,h_walks,h_intentional_walks,h_strikeouts,h_stolen_bases,h_caught_stealing,h_grounded_into_double,h_first_catcher_interference,h_left_on_base,h_pitchers_used,h_individual_earned_runs,h_team_earned_runs,h_wild_pitches,h_balks,h_putouts,h_assists,h_errors,h_passed_balls,h_double_plays,h_triple_plays,hp_umpire_id,hp_umpire_name,1b_umpire_id,1b_umpire_name,2b_umpire_id,2b_umpire_name,3b_umpire_id,3b_umpire_name,lf_umpire_id,lf_umpire_name,rf_umpire_id,rf_umpire_name,v_manager_id,v_manager_name,h_manager_id,h_manager_name,winning_pitcher_id,winning_pitcher_name,losing_pitcher_id,losing_pitcher_name,saving_pitcher_id,saving_pitcher_name,winning_rbi_batter_id,winning_rbi_batter_id_name,v_starting_pitcher_id,v_starting_pitcher_name,h_starting_pitcher_id,h_starting_pitcher_name,v_player_1_id,v_player_1_name,v_player_1_def_pos,v_player_2_id,v_player_2_name,v_player_2_def_pos,v_player_3_id,v_player_3_name,v_player_3_def_pos,v_player_4_id,v_player_4_name,v_player_4_def_pos,v_player_5_id,v_player_5_name,v_player_5_def_pos,v_player_6_id,v_player_6_name,v_player_6_def_pos,v_player_7_id,v_player_7_name,v_player_7_def_pos,v_player_8_id,v_player_8_name,v_player_8_def_pos,v_player_9_id,v_player_9_name,v_player_9_def_pos,h_player_1_id,h_player_1_name,h_player_1_def_pos,h_player_2_id,h_player_2_name,h_player_2_def_pos,h_player_3_id,h_player_3_name,h_player_3_def_pos,h_player_4_id,h_player_4_name,h_player_4_def_pos,h_player_5_id,h_player_5_name,h_player_5_def_pos,h_player_6_id,h_player_6_name,h_player_6_def_pos,h_player_7_id,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,NaN,1,FW1,NaN,1,0,2,54,D,NaN,NaN,NaN,FOR01,200,120,000000000,010010000,30,4,1,0,0,0,0,0,0,1,NaN,6,1,NaN,-1,NaN,4,1,1,1,0,0,27,9,0,3,0,0,31,4,1,0,0,2,0,0,0,1,NaN,0,0,NaN,-1,NaN,3,1,0,0,0,0,27,3,3,1,1,0,boakj901,John Boake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,paboc101,Charlie Pabor,lennb101,Bill Lennon,mathb101,Bobby Mathews,prata101,Al Pratt,NaN,NaN,NaN,NaN,prata101,Al Pratt,mathb101,Bobby Mathews,whitd102,Deacon White,2,kimbg101,Gene Kimball,4,paboc101,Charlie Pabor,7,allia101,Art Allison,8,white104,Elmer White,9,prata101,Al Pratt,1,sutte101,Ezra Sutton,5,carlj102,Jim Carleton,3,bassj101,John Bass,6,selmf101,Frank Sellman,5,mathb101,Bobby Mathews,1,foraj101,Jim Foran,3,goldw101,Wally Goldsmith,6,lennb101,Bill Lennon,2,caret101,Tom Carey,4,mince101,Ed Mincher,7,mcdej101,James McDermott,8,kellb105,Bill Kelly,9,NaN,Y
1,18710505,0,Fri,BS1,NaN,1,WS3,NaN,1,20,18,54,D,NaN,NaN,NaN,WAS01,5000,145,107000435,640113030,41,13,1,2,0,13,0,0,0,18,NaN,5,3,NaN,-1,NaN,12,1,6,6,1,0,27,13,10,1,2,0,49,14,2,0,0,11,0,0,0,10,NaN,2,1,NaN,-1,NaN,14,1,7,7,0,0,27,20,10,2,3,0,dobsh901,Henry Dobson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,wrigh101,Harry Wright,younn801,Nick Young,spala101,Al Spalding,braia102,Asa Brainard,NaN,NaN,NaN,NaN,spala101,Al Spalding,braia102,Asa Brainard,wrigg101,George Wright,6,barnr102,Ross Barnes,4,birdd102,Dave Birdsall,9,mcvec101,Cal McVey,2,wrigh101,Harry Wright,8,goulc101,Charlie Gould,3,schah101,Harry Schafer,5,conef101,Fred Cone,7,spala101,Al Spalding,1,watef102,Fred Waterman,5,forcd101,Davy Force,6,mille105,Everett Mills,3,alli

In [6]:
game_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: object(161)
memory usage: 211.2+ MB


Game log file has over 170,000 records and 161 columns. We will concentrate on the column which are related to the other two tables

In [7]:
park_codes = pd.read_csv("park_codes.csv")
print(park_codes.info())
park_codes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252 entries, 0 to 251
Data columns (total 9 columns):
park_id    252 non-null object
name       252 non-null object
aka        58 non-null object
city       252 non-null object
state      252 non-null object
start      252 non-null object
end        222 non-null object
league     186 non-null object
notes      128 non-null object
dtypes: object(9)
memory usage: 17.8+ KB
None


,park_id,name,aka,city,state,start,end,league,notes
0,ALB01,Riverside Park,NaN,Albany,NY,09/11/1880,05/30/1882,NL,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,NaN,Altoona,PA,04/30/1884,05/31/1884,UA,NaN
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,04/19/1966,NaN,AL,NaN
3,ARL01,Arlington Stadium,NaN,Arlington,TX,04/21/1972,10/03/1993,AL,NaN
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,04/11/1994,NaN,AL,NaN


In [8]:
for i in game_log.columns:
    if i in park_codes.columns:
        print(i)

park_id


Park codes represent different parks where baseball games are held. Park_codes has 9 columns and 252 records. game_log and park_codes have one common column: park_id

In [9]:
person_codes = pd.read_csv("person_codes.csv")
print(person_codes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20494 entries, 0 to 20493
Data columns (total 7 columns):
id              20494 non-null object
last            20494 non-null object
first           20433 non-null object
player_debut    19183 non-null object
mgr_debut       701 non-null object
coach_debut     1566 non-null object
ump_debut       1546 non-null object
dtypes: object(7)
memory usage: 1.1+ MB
None


In [10]:
person_codes.head()

,id,last,first,player_debut,mgr_debut,coach_debut,ump_debut
0,aardd001,Aardsma,David,04/06/2004,NaN,NaN,NaN
1,aaroh101,Aaron,Hank,04/13/1954,NaN,NaN,NaN
2,aarot101,Aaron,Tommie,04/10/1962,NaN,04/06/1979,NaN
3,aased001,Aase,Don,07/26/1977,NaN,NaN,NaN
4,abada001,Abad,Andy,09/10/2001,NaN,NaN,NaN


person_codes has records of 20494 players with 7 columns.

In [11]:
team_codes = pd.read_csv("team_codes.csv")
team_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
team_id      150 non-null object
league       124 non-null object
start        150 non-null int64
end          150 non-null int64
city         150 non-null object
nickname     150 non-null object
franch_id    150 non-null object
seq          150 non-null int64
dtypes: int64(3), object(5)
memory usage: 9.5+ KB


In [12]:
team_codes.head()

,team_id,league,start,end,city,nickname,franch_id,seq
0,ALT,UA,1884,1884,Altoona,Mountain Cities,ALT,1
1,ARI,NL,1998,0,Arizona,Diamondbacks,ARI,1
2,BFN,NL,1879,1885,Buffalo,Bisons,BFN,1
3,BFP,PL,1890,1890,Buffalo,Bisons,BFP,1
4,BL1,NaN,1872,1874,Baltimore,Canaries,BL1,1


team_codes has information about 150 baseball teams with 8 columns

In [13]:
for i in game_log.columns:
    if i in team_codes.columns:
        print(i)
        # no common column

In [14]:
def run_query(q):
    with sqlite3.connect('mlb.db') as conn:
        return pd.read_sql_query(q, conn)

In [15]:
def run_command(c):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level=None
        conn.execute(c)

In [16]:
def show_tables():
    q= """
    Select name, type
    From sqlite_master
    where type in ("table", "view")
    """
    return run_query(q)

In [17]:
def create_table(name, tdata):
    with sqlite3.connect('mlb.db') as conn:
        conn.execute("Drop table if exists {};".format(name))
        tdata.to_sql(name, conn, index=False)

In [18]:
create_table("game_log", game_log)
create_table("person_codes", person_codes)
create_table("team_codes", team_codes)
create_table("park_codes", park_codes)

In [19]:
show_tables()

,name,type
0,game_log,table
1,person_codes,table
2,team_codes,table
3,park_codes,table


In [20]:
c = """
ALTER TABLE game_log ADD COLUMN game_id TEXT;
"""
run_command(c)
c = """
UPDATE game_log SET game_id= (date || h_name || number_of_game)
"""
run_command(c)
q = "Select game_id, date, h_name, number_of_game from game_log limit 5;"
run_query(q)

,game_id,date,h_name,number_of_game
0,18710504FW10,18710504,FW1,0
1,18710505WS30,18710505,WS3,0
2,18710506RC10,18710506,RC1,0
3,18710508CH10,18710508,CH1,0
4,18710509TRO0,18710509,TRO,0


In [21]:
#creating the person table
c = """
CREATE TABLE person (
person_id text primary key,
first_name text,
last_name text
);
"""
run_command(c)

c = """
INSERT INTO person
select id, first, last from person_codes;
"""
run_command(c)

In [25]:
q = "Select * from person limit 5"
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


In [27]:
# creating park table
c1 = """
CREATE TABLE if not exists park (
park_id text primary key,
name text,
nickname text,
city text,
state text,
notes text
);
"""
c2 = """
INSERT OR IGNORE INTO park 
SELECT park_id, name, aka, city, state, notes
FROM park_codes;
"""
q = "Select * from park limit 5;"
run_command(c1)
run_command(c2)
run_query(q)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


In [29]:
# creating the league table
c1 = """
CREATE TABLE IF NOT EXISTS league (
league_id text primary key,
league_name text
);
"""
c2 = """
INSERT OR IGNORE INTO league values 
    ("NL", "National League"),
    ("AL", "American League"),
    ("AA","American Association"),
    ("FL", "Federal League"),
    ("PL", "Players League"),
    ("UA", "Union Association");
"""
q= "Select * from league;"
run_command(c1)
run_command(c2)
run_query(q)

,league_id,league_name
0,NL,National League
1,AL,American League
2,AA,American Association
3,FL,Federal League
4,PL,Players League
5,UA,Union Association


In [30]:
appearancet = pd.read_csv("appearance_type.csv")
appearancet.head()

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [31]:
# creating the appearance type table
name = 'appearance_type'
with sqlite3.connect('mlb.db') as conn:
    conn.execute("Drop table if exists {};".format(name))
    appearancet.to_sql(name, conn, index=False)

In [32]:
q = "select * from appearance_type limit 5;"
run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense


In [36]:
# creating the team table
c1 = """
CREATE TABLE if not exists team (
    team_id text primary key,
    league_id text,
    city text,
    nickname text,
    franch_id text,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
"""
c2 = """
INSERT or IGNORE INTO team
Select team_id, league, city, nickname, franch_id
FROM team_codes;
"""
q= "Select * from team limit 5;"
run_command(c1)
run_command(c2)
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [40]:
# creating the GAME table
c1 = """
CREATE TABLE game (
    game_id text primary key,
    date text,
    number_of_game integer,
    park_id text,
    length_outs integer,
    day boolean,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    length_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
"""
c2 = """
INSERT OR IGNORE INTO game
SELECT 
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE 
        WHEN day_night = 'd' THEN 1
        WHEN day_night = 'N' THEN 0
        ELSE NULL
    END AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
"""
q= "select * from game limit 5;"
run_command(c1)
run_command(c2)
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,length_minutes,additional_info,acquisition_info
0,18710504FW10,18710504,0,FOR01,54,None,None,None,None,200,120,None,Y
1,18710505WS30,18710505,0,WAS01,54,None,None,None,None,5000,145,HTBF,Y
2,18710506RC10,18710506,0,RCK01,54,None,None,None,None,1000,140,None,Y
3,18710508CH10,18710508,0,CHI01,54,None,None,None,None,5000,150,None,Y
4,18710509TRO0,18710509,0,TRO01,54,None,None,None,None,3250,145,HTBF,Y


In [48]:
# creating the team table
c1 = """
CREATE TABLE if not exists team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id)
);
"""
c2 = """
INSERT INTO team_appearance 
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
"""
q = """
SELECT * FROM team_appearance
WHERE game_id = (
                 SELECT MIN(game_id) from game
                )
   OR game_id = (
                 SELECT MAX(game_id) from game
                )
ORDER By game_id, home;
"""
run_command(c1)
run_command(c2)
run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,CL1,18710504FW10,0,None,0,000000000,30,4,1,0,0,0,0,0,0,1,NaN,6,1,NaN,-1,NaN,4,1,1,1,0,0,27,9,0,3,0,0
1,FW1,18710504FW10,1,None,2,010010000,31,4,1,0,0,2,0,0,0,1,NaN,0,0,NaN,-1,NaN,3,1,0,0,0,0,27,3,3,1,1,0
2,MIA,20161002WAS0,0,NL,7,000230020,38,14,1,1,2,7,1,0,0,3,2.0,10,1,1.0,1,0.0,8,7,10,10,1,0,24,11,0,0,1,0
3,WAS,20161002WAS0,1,NL,10,03023002x,30,10,2,0,1,10,1,1,1,8,0.0,3,2,0.0,1,0.0,7,6,7,7,1,0,27,11,0,0,1,0


In [55]:
#print(run_query("select sql from sqlite_master where name='game_log' and type='table'")['sql'][0])